<a href="https://colab.research.google.com/github/ShoaibSajid/Python_CNN/blob/main/CNN_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Importing libraries

from logging import raiseExceptions
import os
import pickle
from tqdm import tqdm
# import mnist
import numpy as np
from conv import Conv3x3, Conv3x3_n_to_n_padding, Conv3x3_1_to_n_padding
from maxpool import MaxPool2
from softmax import Softmax
from relu import Relu
from softmax_test import Softmax_test
from fc import FC

# Define settings for run

debug=False

shuffle_data=False

run_train=True
run_val=True

load_saved_weights=False
weight_file='weights/best_99.pkl'

total_epoch=100
training_acc_internal=1000

# Initialize the network layers

conv0   = Conv3x3_1_to_n_padding( output=8                        )     # 28x28x1   -> 28x28x8  (Convolution with 8 filters)
pool0   = MaxPool2              (                                 )     # 28x28x8   -> 14x14x8  (MaxPooling 2x2)
conv1   = Conv3x3_n_to_n_padding( output=16     ,   input=8       )     # 14x14x8   -> 14x14x16 (Convolution with 8 filters)
pool1   = MaxPool2              (                                 )     # 14x14x16  -> 07x07x16 (MaxPooling 2x2)
# conv2   = Conv3x3_n_to_n_padding( output=32     ,   input=16      )
# conv3   = Conv3x3_n_to_n_padding( output=64     ,   input=32      )
fc0     = FC                    ( 7 * 7 * 16  ,   7 * 7 * 16      )     # 784       -> 784      (FC)
fc1     = FC                    ( 7 * 7 * 16  ,   10              )     # 784       -> 10       (FC)
softmax = Softmax               (                                 )     # 13x13x8   -> 10       (Softmax)
relu    = Relu                  (                                 )     # 13x13x8   -> 10       (Softmax)

In [20]:
# Defining the network - Forward propagation
def forward(im, label='noneed', debug=False):
    im        = (im / 255) - 0.5

    # Conv 0 with Pool
    out_conv0 = conv0.forward   ( im            )
    out_pool0 = pool0.forward   ( out_conv0     )

    # Conv 1 with Pool
    out_conv1 = conv1.forward   ( out_pool0     )
    out_pool1 = pool1.forward   ( out_conv1     )

    # Swap axes to realign for flattening
    out_pool2 = np.swapaxes(out_pool1,0,2)
    out_pool3 = np.swapaxes(out_pool2,1,2)

    # FC0 and Relu
    out_fc0   = fc0.forward     ( out_pool3     )

    # FC1 and SoftMax
    out_fc1   = fc1.forward     ( out_fc0       )
    out_soft  = softmax.forward ( out_fc1       )

    if debug:
        print(f"Input Image: {im[-4]}\n")
        print(f"x_conv0 filters : {conv0.filters[0]}\n")
        print(f"x_conv0 : {out_conv0[:,:,0][-1]}\n")
        print(f"MaxPool0: {out_pool0[:,:,0][-1]}\n")
        print(f"x_conv1 filters : {conv1.filters[0,:,:,0]}\n")
        print(f"x_conv1 : {out_conv1[:,:,0][-1]}\n")
        print(f"MaxPool1: {out_pool1[:,:,0][-1]}\n")
        print(f"FC0 Weights: {fc0.weights[:,0][:10]}\n")
        print(f"FC0 output: {out_fc0[:10]}\n")
        print(f"FC1 Weights: {fc1.weights[:,0][:10]}\n")
        print(f"FC1 output: {out_fc1[:10]}\n")
        print(f"SoftMax output: {out_soft}\n")

    return out_soft

In [1]:
def load_weights(name):
  if os.path.isfile(name): 
    weight_file = open(str(name), "rb")
    weights = pickle.load(weight_file)
    conv0.filters  = weights["conv0"]      
    conv1.filters  = weights["conv1"]      
    fc0.weights    = weights["fc0_weights"]
    fc0.biases     = weights["fc0_biases" ]
    fc1.weights    = weights["fc1_weights"]
    fc1.biases     = weights["fc1_biases" ]
    lr             = weights["lr" ]
    max_acc        = weights["max_acc"]
    print(f"\nLoading weights from {name} file. LR restored to {lr}. Last Accuracy {max_acc}%")
    return lr, max_acc
  else:
    print("Weights file not found.")
    lr=0.005
    max_acc=0
    return lr, max_acc
   

lr, max_acc = load_weights(weight_file)


Loading weights from weights/best_99.pkl file. LR restored to 0.0005. Last Accuracy 99%


In [66]:
# load samples from MNIST test dataset
import torchvision
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, ToPILImage

data_folder = '../MNIST'
tedt = datasets.MNIST(
    root=data_folder,
    train=False,
    download=True,
    transform=ToTensor(),
)

tedl = DataLoader(tedt, batch_size=1)

trit = iter(tedl)


In [111]:
x,y = next(trit)

# tensor(0-1) > pil(0-255) > np(0-255)
topil = torchvision.transforms.ToPILImage()
x = x.reshape(1,28,28)

xpil = topil(x)
xnp  = np.array(xpil)

pre = forward(xnp).argmax()

In [101]:
print('prediction',pre, 'gt',y)
xpil

prediction 7 gt tensor([7])


In [105]:
print('prediction',pre, 'gt',y)
xpil

prediction 2 gt tensor([2])


In [108]:
print('prediction',pre, 'gt',y)
xpil

prediction 1 gt tensor([1])


In [110]:
print('prediction',pre, 'gt',y)
xpil

prediction 0 gt tensor([0])


In [112]:
print('prediction',pre, 'gt',y)
xpil

prediction 4 gt tensor([4])
